In [77]:
''' GENERAL '''
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import json
import re

''' ANALYSIS '''
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

''' VISUALISATION '''
import statsmodels.api as sm

In [53]:
def write_table(table, name, caption, label, dec=None, sig_note=False, index=True):
    if dec:
        dec = f'%.{dec}f'

    table_latex = table.to_latex(caption=caption, label='tab:' + label, float_format=dec, index=index)
    table_latex = table_latex.replace('\\begin{table}\n', '\\begin{table}\n\\centering\n')

    sig_note_str = '''\\bottomrule
\\addlinespace[1ex]
\\multicolumn{3}{l}
    {
        \\textsuperscript{***}$p<0.01$,
        \\textsuperscript{**}$p<0.05$,
        \\textsuperscript{*}$p<0.1$
    }'''

    if sig_note:
        table_latex = table_latex.replace('\\bottomrule', sig_note_str)

    with open(f'../tables/{name}.tex', 'w') as out:
        out.write(table_latex)

    print(table_latex)

In [54]:
def add_sigs(val):
    if val <= 0.01:
        return str(val) + '***'
    elif 0.01 < val <= 0.05:
        return str(val) + '**'
    elif 0.05 < val <= 0.1:
        return str(val) + '*'
    else:
        return str(val)

# PREP

In [55]:
''' COUNTS '''

sim_occ = pd.read_csv('sim_occ_level_3.csv', dtype={'sim_occ': 'str'})
counts_long = pd.DataFrame(sim_occ.groupby('app_year', as_index=False)['sim_occ'].value_counts())

def rel_count(row, nr_patents):
    count = row['count']
    year = row['app_year']
    total = nr_patents.loc[nr_patents['app_year'] == year, 'count'].iloc[0]

    return (count / total) * 100

patents_per_year = pd.DataFrame(sim_occ['app_year'].value_counts())
patents_per_year = patents_per_year.sort_values('app_year', axis=0).reset_index()

counts_long['count_rel'] = counts_long.apply(lambda x: rel_count(x, patents_per_year), axis=1)

counts_wide = counts_long.query("2009 <= app_year <= 2020")
counts_wide = counts_wide.pivot(columns='sim_occ', index='app_year', values='count')
counts_wide.columns.name = None
counts_wide.index.name = None
counts_wide = counts_wide.fillna(0)
counts_wide = counts_wide.astype('int')

counts_total = counts_long.groupby('sim_occ')['count'].sum().sort_values(ascending=False)

In [56]:
''' LABOR FORCE SURVEY '''
# Labour force survey
lfs_long = pd.read_csv('lfs.csv', dtype={'ISCO3D': 'str'})
lfs_long.rename({'nuts': 'nuts_code', 'ISCO3D': 'isco_code', 'NACE1D': 'nace_code'}, axis=1, inplace=True)

lfs_long['nuts_code'] = lfs_long['nuts_code'].apply(lambda x: x[:2])
lfs_long = lfs_long.groupby(['nuts_code', 'year', 'isco_code'], as_index=False)['n'].sum()

# Add 0 to military isco codes: "21" --> "021"
lfs_long['isco_code'] = np.select(
    condlist=[
        lfs_long['isco_code'].apply(lambda x: len(x) == 2),
        lfs_long['isco_code'].apply(lambda x: len(x) == 1)
    ],
    choicelist=[
        lfs_long['isco_code'].apply(lambda x: '0' + x),
        lfs_long['isco_code'].apply(lambda x: '00' + x),
    ],
    default=lfs_long['isco_code']
)
lfs_long.set_index(['nuts_code', 'year'], inplace=True)

lfs_na = ['BG', 'MT', 'PL', 'SI', 'SK'] # Countries with missing lfs data
lfs_long = lfs_long.drop(lfs_na)

''' pivot '''

lfs_wide = lfs_long.pivot_table(index=['nuts_code', 'year'], columns='isco_code', values='n')
lfs_wide = lfs_wide.fillna(0)
lfs_wide =  lfs_wide.astype('int')

lfs_wide = lfs_wide.reset_index()
lfs_wide.set_index(['nuts_code', 'year'], inplace=True)
lfs_wide.columns.name = None

lfs_wide = lfs_wide.drop(columns=[c for c in lfs_wide.columns if not c in counts_wide.columns])

''' aggregated '''

lfs_wide_eu = lfs_wide.groupby('year').mean()
# lfs_wide_eu['nuts_code'] = 'EU'
# lfs_wide_eu =lfs_wide_eu.set_index('nuts_code', append=True)
# lfs_wide_eu = lfs_wide_eu.reorder_levels(['nuts_code', 'year'])

In [57]:
''' RENEWABLE ENERGY '''

ren_energy_wide = pd.read_csv('estat_nrg_ind_ren.tsv', sep='\t')

ren_energy_cols = ['freq', 'nrg_bal', 'unit', 'geo']
ren_energy_wide[ren_energy_cols] = ren_energy_wide.iloc[:, 0].str.split(',', expand=True)

ren_energy_cols = ren_energy_wide.columns.tolist()
ren_energy_cols = ren_energy_cols[-4:] + ren_energy_cols[1:-4]
ren_energy_cols = ren_energy_cols[3:]

ren_energy_wide = ren_energy_wide.loc[ren_energy_wide['nrg_bal'] == 'REN', ren_energy_cols]

ren_energy_wide.columns = [int(c) if not c.isalpha() else c for c in ren_energy_wide.columns]
ren_energy_wide.replace(': ', np.nan, inplace=True)

ren_energy_wide[ren_energy_wide.columns[1:]] = ren_energy_wide[ren_energy_wide.columns[1:]].astype('float')

# ren_energy.drop(ren_energy[ren_energy['geo'].isin(['GE', 'BA'])].index, inplace=True)
ren_energy_wide.reset_index(drop=True, inplace=True)
ren_energy_wide.rename(columns={'geo': 'nuts_code'}, inplace=True)

ren_energy_wide.drop(columns=range(2004, 2013), inplace=True)
ren_energy_wide.drop(columns=range(2021, 2023), inplace=True)

ren_energy_wide = ren_energy_wide.set_index('nuts_code')
# ren_energy.drop('EU27_2020', inplace=True)

ren_energy_long = ren_energy_wide.reset_index().melt(id_vars=['nuts_code'], var_name='year', value_name='ren_energy').sort_values(['nuts_code', 'year']).set_index(['nuts_code', 'year'])

ren_energy_long_eu = ren_energy_long.loc['EU27_2020']
scaler = StandardScaler()
ren_energy_long_eu_s = pd.DataFrame(scaler.fit_transform(ren_energy_long_eu))
ren_energy_long_eu_s.index = range(2013, 2021)
ren_energy_long_eu_s.columns = ['ren_en']

In [58]:
ren_idx = set(ren_energy_long.index.levels[0])
lfs_idx = set(lfs_wide.index.levels[0])

ren_energy_long.drop(list(ren_idx - lfs_idx), inplace=True)
lfs_wide.drop(list(lfs_idx - ren_idx), inplace=True)
lfs_long.drop(list(lfs_idx - ren_idx), inplace=True)

ren_energy_long.index = ren_energy_long.index.remove_unused_levels()
lfs_wide.index = lfs_wide.index.remove_unused_levels()
lfs_long.index = lfs_long.index.remove_unused_levels()

In [59]:
''' R&D '''

rd_data = pd.read_csv('COUNTRY_BUDGETS_SUMMARY.csv', sep=';', header=4)
rd_data = rd_data.replace('..', np.nan)
rd_data['VALUE'] = rd_data['VALUE'].astype('float')
rd_data = rd_data.query("PRODUCT == 'RDDEURO' and (2013 <= TIME <= 2020) and FLOW == 'TOTAL'")
rd_data = rd_data[['TIME', 'COUNTRY', 'VALUE']]
rd_data.columns = ['year', 'nuts_code', 'rnd']
rd_data = rd_data.sort_values('nuts_code')
rd_data = rd_data.set_index(['nuts_code', 'year'])

In [150]:
''' ISCO '''

# Read file
isco = pd.read_excel('isco.xlsx', dtype={'ISCO 08 Code': 'str'})
# Rename columns
isco.rename(columns={'Level': 'isco_level', 'ISCO 08 Code': 'isco_code', 'Title EN': 'isco_title', 'Tasks include': 'isco_tasks', 'Definition': 'isco_definition'}, inplace=True)
# Drop unnecessary columns
isco = isco.iloc[:, :5]
# Add '0'/'00' to level 1/2 isco codes ('21' --> '210', '8' --> '800')
isco['isco_code'] = np.select(
    condlist=[
        isco['isco_level'] == 1,
        isco['isco_level'] == 2
    ],
    choicelist=[
        isco['isco_code'].apply(lambda x: x + '00'),
        isco['isco_code'].apply(lambda x: x + '0'),
    ],
    default=isco['isco_code']
)

# Analysis

In [60]:
''' Variance Inflation Factor '''

# Calculate VIF for each independent variable
vif_data = pd.DataFrame()
vif_data["feature"] = lfs_wide.columns
vif_data["VIF"] = [variance_inflation_factor(lfs_wide.values, i) for i in range(len(lfs_wide.columns))]

# vif_data.describe()

In [61]:
def weight_with_patent_lag(x:"pd.Series", counts:"Wide"):
        # Series as float
        x = x.astype('float')
        # Iterate over series
        for year in x.index:
            # LFS value for year y
            lfs_val = x.loc[year]
            # Lagged sum of occupation
            sum_ = counts.loc[year-4:year, x.name].sum()
            # Lagged total amount of occupations
            total = counts.loc[year-4:year].sum(axis=1).sum()
            # Sum of occupation relative to total
            rel = sum_/total
            # Assign relative value to x
            x.loc[year] = lfs_val * rel

        return x

def scale_and_weight(lfs:"Series wide"):
    scaler = StandardScaler()
    lfs_scaled = lfs.copy().astype('float')
    lfs_scaled.loc[:, :] = scaler.fit_transform(lfs)
    lfs_scaled_weighted = lfs_scaled.apply(lambda x: weight_with_patent_lag(x, counts_wide), axis=0)

    return lfs_scaled_weighted

def optimal_nr_comp(lfs: "Wide", ren_energy: "Long"):
    n_components = min(lfs.shape)
    mse_scores = []

    # Perform PCA and cross-validation for each number of components
    for n in range(1, n_components + 1):
        pca = PCA(n_components=n)
        X_pca = pca.fit_transform(lfs)

        model = LinearRegression()
        scores = cross_val_score(model, X_pca, ren_energy, cv=5, scoring='neg_mean_squared_error')
        mse_scores.append(-scores.mean())

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    # Plot the MSE scores
    plt.plot(range(1, n_components + 1), mse_scores, marker='o')
    plt.xlabel('Number of Principal Components')#, fontsize=16)
    plt.ylabel('Mean Squared Error')#, fontsize=16)
    plt.tick_params(axis='both')#, labelsize=14)  # Change the font size for major ticks

    plt.subplot(1, 2, 2)
    pca = PCA().fit(lfs)
    plt.plot(pd.DataFrame(np.cumsum(pca.explained_variance_ratio_), index=range(1,8)))
    plt.xlabel('Number of Components')#, fontsize=16)
    plt.ylabel('Cumulative Explained Variance')#, fontsize=16)
    plt.tick_params(axis='both')#, labelsize=14)  # Change the font size for major ticks

In [114]:
''' OLS ANALYSIS '''

def ols_pca(lfs, ren_energy, rd_data):
    # Init scaler object
    scaler = StandardScaler()
    # Scale and weight lfs data
    lfs_transformed = scale_and_weight(lfs)
    # Create principal components with given n components
    pca = PCA(n_components=1)
    components = pca.fit_transform(lfs_transformed)
    # Scale R&D data
    rd_scaled = scaler.fit_transform(rd_data)
    # Scale renewable energy data
    ren_energy_scaled = scaler.fit_transform(ren_energy)

    # Create dataframe
    df = pd.DataFrame(components)
    df['R&D'] = rd_scaled
    df['ren_energy'] = ren_energy_scaled
    df.index = range(2013,2021)
    cols = df.columns.tolist()
    cols[0] = 'PC'
    df.columns = cols

    df = df.drop(2013)

    X = df.drop(columns='ren_energy')
    X = sm.add_constant(X)

    Y = df['ren_energy']

    model = sm.OLS(Y, X).fit()

    return model

In [115]:
cols = [c for c in lfs_wide.columns if re.match('7..', c)]
lfs_wide_sig = lfs_wide[cols]

In [116]:
pvals_df = pd.DataFrame()
models = {}
countries = lfs_wide.index.levels[0]

for c in countries:
    model = ols_pca(lfs_wide_sig.loc[c],
                    rd_data.loc['EU'],
                    ren_energy_long.loc[c])

    models[c] = model

In [117]:
df_ols = pd.DataFrame()
for c in models.keys():
    df = pd.DataFrame()
    model = models[c]
    df['coef'] = model.params
    df['std err'] = model.bse
    df['t'] = model.tvalues
    df['p-value'] = model.pvalues
    df.index.name = 'stats'
    df['country'] = c
    df = df.set_index('country', append=True)
    df = df.reorder_levels(['country', 'stats'])
    df_ols = pd.concat([df_ols, df])

df_ols = df_ols.apply(lambda x: round(x, 4))
df_ols['p-value'] = df_ols['p-value'].apply(add_sigs)

In [70]:
# write_table(df_ols[36:], 'ols_eu', 'Regression results per country', 'ols_eu', index=True, dec=4, sig_note=True)

In [160]:
isco_def_table = isco[isco['isco_code'].isin(counts_total.index[:30])][['isco_code', 'isco_title']]
isco_def_table.columns = ['ISCO Code', 'Definition']
write_table(isco_def_table, 'isco_def', 'ISCO Titles (Top 30 recurring occupations', 'isco_def', index=False)

\begin{table}
\centering
\caption{ISCO Titles (Top 30 recurring occupations}
\label{tab:isco_def}
\begin{tabular}{ll}
\toprule
ISCO Code & Definition \\
\midrule
211 & Physical and Earth Science Professionals \\
212 & Mathematicians, Actuaries and Statisticians \\
215 & Electrotechnology Engineers \\
226 & Other Health Professionals \\
251 & Software and Applications Developers and Analysts \\
311 & Physical and Engineering Science Technicians \\
313 & Process Control Technicians \\
314 & Life Science Technicians and Related Associate Professionals \\
352 & Telecommunications and Broadcasting Technicians \\
711 & Building Frame and Related Trades Workers \\
712 & Building Finishers and Related Trades Workers \\
713 & Painters, Building Structure Cleaners and Related Trades Workers \\
721 & Sheet and Structural Metal Workers, Moulders and Welders, and Related Workers \\
731 & Handicraft Workers \\
732 & Printing Trades Workers \\
741 & Electrical Equipment Installers and Repairers \\
74